In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import tensorflow as tf
import os

In [57]:
from sktime.forecasting.model_selection import (
    ForecastingGridSearchCV,
    SlidingWindowSplitter,
    temporal_train_test_split,
)

In [21]:
import warnings
warnings.filterwarnings('ignore')

In [17]:
from datetime import datetime

In [165]:
raw_col = ['date', 'highest', 'high', 'mid', 'low', 'ave', 'ave_dif', 'volume', 'vol_dif', 'resid']
raw = pd.read_csv('FS443.csv',names=raw_col).drop(['ave_dif', 'vol_dif', 'resid'], axis=1)
for i in range(raw.shape[0]):
    raw.volume[i] = int(raw.volume[i].split(' ')[0].replace(',', ''))
raw.volume = raw.volume.values.astype('int')

ts_date = raw.copy().date
for itr in range(raw.shape[0]):
    date = raw.date[itr].split('/')
    date[0] = str(int(date[0]) + 1911)
    result = '/'.join(date)
    ts_date[itr] =  np.datetime64(datetime.strptime(result, "%Y/%m/%d").date())
    


ts = raw.copy()
ts.date = ts_date
ts['date'] = pd.to_datetime(ts['date'], errors='coerce')
ts = ts.set_index('date')
ts.index = pd.to_datetime(ts.index)
ts.index = ts.index.to_period('D') 

ts

,highest,high,mid,low,ave,volume
date,,,,,,
1998-12-01,195,131,77,59,84.0,327
1998-12-02,165,144,111,93,113.0,456
1998-12-03,204,160,82,65,94.0,432
1998-12-04,189,161,114,80,116.0,583
1998-12-06,181,157,90,55,96.0,1052
...,...,...,...,...,...,...
2020-11-26,201,201,137,67,134.9,97
2020-11-27,170,170,112,52,111.8,176
2020-11-28,171,171,170,152,166.7,81


In [ ]:
temporal_train_test_split(ts, test_size=36)

In [166]:
n_past = 30
n_future = 14
n_features = 6

In [167]:
ts

,highest,high,mid,low,ave,volume
date,,,,,,
1998-12-01,195,131,77,59,84.0,327
1998-12-02,165,144,111,93,113.0,456
1998-12-03,204,160,82,65,94.0,432
1998-12-04,189,161,114,80,116.0,583
1998-12-06,181,157,90,55,96.0,1052
...,...,...,...,...,...,...
2020-11-26,201,201,137,67,134.9,97
2020-11-27,170,170,112,52,111.8,176
2020-11-28,171,171,170,152,166.7,81


In [168]:
train,final_test_df = ts[:-(n_past+n_future)], ts[-(n_past+n_future):] 

In [170]:
train_df, test_df = temporal_train_test_split(train, test_size=0.3)

In [171]:
train_df

,highest,high,mid,low,ave,volume
date,,,,,,
1998-12-01,195,131,77,59,84.0,327
1998-12-02,165,144,111,93,113.0,456
1998-12-03,204,160,82,65,94.0,432
1998-12-04,189,161,114,80,116.0,583
1998-12-06,181,157,90,55,96.0,1052
...,...,...,...,...,...,...
2014-02-19,170,146,126,88,122.0,447
2014-02-20,177,165,97,67,104.0,230
2014-02-21,150,146,123,67,116.0,379


In [172]:
final_test_df

,highest,high,mid,low,ave,volume
date,,,,,,
2020-10-09,171,170,103,74,110.8,476
2020-10-12,173,170,116,76,118.5,741
2020-10-13,171,159,101,55,103.4,674
2020-10-14,144,127,83,50,84.9,473
2020-10-15,125,115,63,51,71.2,494
2020-10-16,105,83,68,62,69.6,480
2020-10-17,113,95,81,75,82.4,360
2020-10-19,140,140,124,82,118.5,174
2020-10-20,185,183,128,106,134.4,136


In [173]:
test_df

,highest,high,mid,low,ave,volume
date,,,,,,
2014-02-25,170,152,130,89,126.0,545
2014-02-26,156,155,130,87,126.0,836
2014-02-27,185,179,134,89,133.0,391
2014-02-28,182,179,135,116,139.0,630
2014-03-01,161,151,126,84,122.0,745
...,...,...,...,...,...,...
2020-10-02,225,225,172,129,174.1,165
2020-10-05,220,220,174,114,171.0,251
2020-10-06,218,201,153,125,157.4,208


In [174]:
train = train_df.copy()
scalers={}
for i in train_df.columns:
    scaler = MinMaxScaler(feature_range=(0,1))
    s_s = scaler.fit_transform(train[i].values.reshape(-1,1))
    s_s=np.reshape(s_s,len(s_s))
    scalers['scaler_'+ i] = scaler
    train[i]=s_s
test = test_df.copy()
for i in train_df.columns:
    scaler = scalers['scaler_'+i]
    s_s = scaler.transform(test[i].values.reshape(-1,1))
    s_s=np.reshape(s_s,len(s_s))
    scalers['scaler_'+i] = scaler
    test[i]=s_s
    
final_test = final_test_df.copy()
for i in train_df.columns:
    scaler = scalers['scaler_'+i]
    s_s = scaler.transform(final_test[i].values.reshape(-1,1))
    s_s=np.reshape(s_s,len(s_s))
    scalers['scaler_'+i] = scaler
    final_test[i]=s_s

In [176]:
final_test

,highest,high,mid,low,ave,volume
date,,,,,,
2020-10-09,0.394009,0.427136,0.290960,0.240260,0.320231,0.092718
2020-10-12,0.398618,0.427136,0.327684,0.246753,0.342486,0.145444
2020-10-13,0.394009,0.399497,0.285311,0.178571,0.298844,0.132113
2020-10-14,0.331797,0.319095,0.234463,0.162338,0.245376,0.092121
2020-10-15,0.288018,0.288945,0.177966,0.165584,0.205780,0.096299
2020-10-16,0.241935,0.208543,0.192090,0.201299,0.201156,0.093514
2020-10-17,0.260369,0.238693,0.228814,0.243506,0.238150,0.069638
2020-10-19,0.322581,0.351759,0.350282,0.266234,0.342486,0.032630
2020-10-20,0.426267,0.459799,0.361582,0.344156,0.388439,0.025070


In [162]:
final_test

,highest,high,mid,low,ave,volume
date,,,,,,
2020-10-09,-0.211982,-0.145729,-0.418079,-0.519481,-0.199533,-0.814564
2020-10-12,-0.202765,-0.145729,-0.344633,-0.506494,-0.143905,-0.709113
2020-10-13,-0.211982,-0.201005,-0.429379,-0.642857,-0.252994,-0.735774
2020-10-14,-0.336406,-0.361809,-0.531073,-0.675325,-0.386646,-0.815758
2020-10-15,-0.423963,-0.422111,-0.644068,-0.668831,-0.485620,-0.807402
2020-10-16,-0.516129,-0.582915,-0.615819,-0.597403,-0.497179,-0.812973
2020-10-17,-0.479263,-0.522613,-0.542373,-0.512987,-0.404707,-0.860724
2020-10-19,-0.354839,-0.296482,-0.299435,-0.467532,-0.143905,-0.934739
2020-10-20,-0.147465,-0.080402,-0.276836,-0.311688,-0.029036,-0.949861


In [97]:
def split_series(series, n_past, n_future):
  #
  # n_past ==> no of past observations
  #
  # n_future ==> no of future observations 
  #
  X, y = list(), list()
  for window_start in range(len(series)):
    past_end = window_start + n_past
    future_end = past_end + n_future
    if future_end > len(series):
      break
    # slicing the past and future parts of the window
    past, future = series[window_start:past_end, :], series[past_end:future_end, :]
    X.append(past)
    y.append(future)
  return np.array(X), np.array(y)

In [177]:
X_train, y_train = split_series(train.values,n_past, n_future)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1],n_features))
y_train = y_train.reshape((y_train.shape[0], y_train.shape[1], n_features))

X_test, y_test = split_series(test.values,n_past, n_future)
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1],n_features))
y_test = y_test.reshape((y_test.shape[0], y_test.shape[1], n_features))

X_final_test, y_final_test = split_series(final_test.values,n_past, n_future)
X_final_test = X_final_test.reshape((X_final_test.shape[0], X_final_test.shape[1],n_features))
y_final_test = y_final_test.reshape((y_final_test.shape[0], y_final_test.shape[1], n_features))

In [179]:
# E1D1
# n_features ==> no of features at each timestep in the data.
#
encoder_inputs = tf.keras.layers.Input(shape=(n_past, n_features))
encoder_l1 = tf.keras.layers.LSTM(100, return_state=True)
encoder_outputs1 = encoder_l1(encoder_inputs)

encoder_states1 = encoder_outputs1[1:]

#
decoder_inputs = tf.keras.layers.RepeatVector(n_future)(encoder_outputs1[0])

#
decoder_l1 = tf.keras.layers.LSTM(100, return_sequences=True)(decoder_inputs,initial_state = encoder_states1)
decoder_outputs1 = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(n_features))(decoder_l1)

#
model_e1d1 = tf.keras.models.Model(encoder_inputs,decoder_outputs1)

#
model_e1d1.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 30, 6)]      0                                            
__________________________________________________________________________________________________
lstm_6 (LSTM)                   [(None, 100), (None, 42800       input_3[0][0]                    
__________________________________________________________________________________________________
repeat_vector_2 (RepeatVector)  (None, 14, 100)      0           lstm_6[0][0]                     
__________________________________________________________________________________________________
lstm_7 (LSTM)                   (None, 14, 100)      80400       repeat_vector_2[0][0]            
                                                                 lstm_6[0][1]               

In [ ]:
n_timesteps, n_features, n_outputs = train_x.shape[1], train_x.shape[2], train_y.shape[1]
# reshape into subsequences [samples, time steps, rows, cols, channels]
train_x = train_x.reshape((train_x.shape[0], n_steps, 1, n_length, n_features))
# reshape output into [samples, timesteps, features]
train_y = train_y.reshape((train_y.shape[0], train_y.shape[1], 1))
# define model
model = Sequential()
model.add(ConvLSTM2D(filters=64, kernel_size=(1,3), activation='relu', input_shape=(n_steps, 1, n_length, n_features)))
model.add(Flatten())
model.add(RepeatVector(n_outputs))
model.add(LSTM(200, activation='relu', return_sequences=True))
model.add(TimeDistributed(Dense(100, activation='relu')))
model.add(TimeDistributed(Dense(1)))
model.compile(loss='mse', optimizer='adam')
# fit network
model.fit(train_x, train_y, epochs=epochs, batch_size=batch_size, verbose=verbose)


In [180]:
# E2D2
# n_features ==> no of features at each timestep in the data.
#
encoder_inputs = tf.keras.layers.Input(shape=(n_past, n_features))
encoder_l1 = tf.keras.layers.LSTM(100,return_sequences = True, return_state=True)
encoder_outputs1 = encoder_l1(encoder_inputs)
encoder_states1 = encoder_outputs1[1:]
encoder_l2 = tf.keras.layers.LSTM(100, return_state=True)
encoder_outputs2 = encoder_l2(encoder_outputs1[0])
encoder_states2 = encoder_outputs2[1:]
#
decoder_inputs = tf.keras.layers.RepeatVector(n_future)(encoder_outputs2[0])
#
decoder_l1 = tf.keras.layers.LSTM(100, return_sequences=True)(decoder_inputs,initial_state = encoder_states1)
decoder_l2 = tf.keras.layers.LSTM(100, return_sequences=True)(decoder_l1,initial_state = encoder_states2)
decoder_outputs2 = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(n_features))(decoder_l2)
#
model_e2d2 = tf.keras.models.Model(encoder_inputs,decoder_outputs2)
#
model_e2d2.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 30, 6)]      0                                            
__________________________________________________________________________________________________
lstm_8 (LSTM)                   [(None, 30, 100), (N 42800       input_4[0][0]                    
__________________________________________________________________________________________________
lstm_9 (LSTM)                   [(None, 100), (None, 80400       lstm_8[0][0]                     
__________________________________________________________________________________________________
repeat_vector_3 (RepeatVector)  (None, 14, 100)      0           lstm_9[0][0]                     
____________________________________________________________________________________________

In [181]:
reduce_lr = tf.keras.callbacks.LearningRateScheduler(lambda x: 1e-3 * 0.90 ** x)
model_e1d1.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.Huber())
history_e1d1=model_e1d1.fit(X_train,y_train,epochs=25,validation_data=(X_test,y_test),batch_size=32,verbose=0,callbacks=[reduce_lr])


In [182]:
model_e2d2.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.Huber())
history_e2d2=model_e2d2.fit(X_train,y_train,epochs=25,validation_data=(X_test,y_test),batch_size=32,verbose=0,callbacks=[reduce_lr])

In [118]:
pred_e1d1.shape

(1, 14, 6)

In [223]:
pred_e1d1=model_e1d1.predict(X_final_test)
pred_e2d2=model_e2d2.predict(X_final_test)

In [224]:
y_final = y_final_test.copy()

In [205]:
y_final

array([[[0.34331797, 0.31407035, 0.20903955, 0.18831169, 0.23323699,
         0.07739753],
        [0.38940092, 0.39447236, 0.32485876, 0.26298701, 0.33699422,
         0.0356148 ],
        [0.39170507, 0.35427136, 0.30508475, 0.16883117, 0.29855491,
         0.05789893],
        [0.32258065, 0.31658291, 0.24858757, 0.25      , 0.27052023,
         0.05650617],
        [0.37096774, 0.36180905, 0.26836158, 0.24350649, 0.29046243,
         0.02725826],
        [0.32718894, 0.35678392, 0.33333333, 0.24025974, 0.32861272,
         0.04257859],
        [0.36866359, 0.40201005, 0.39830508, 0.34415584, 0.39797688,
         0.02845205],
        [0.39400922, 0.42211055, 0.42655367, 0.26948052, 0.4066474 ,
         0.03322722],
        [0.40552995, 0.44221106, 0.48022599, 0.52272727, 0.48959538,
         0.01830481],
        [0.46313364, 0.50502513, 0.38700565, 0.21753247, 0.38988439,
         0.01730999],
        [0.39170507, 0.42713568, 0.31638418, 0.16883117, 0.32312139,
         0.03302825],

In [225]:
for index,i in enumerate(train_df.columns):
    scaler = scalers['scaler_'+i]
#     pred1_e1d1[:,:,index]=scaler.inverse_transform(pred1_e1d1[:,:,index])
    pred_e1d1[:,:,index]=scaler.inverse_transform(pred_e1d1[:,:,index])
    
#     pred1_e2d2[:,:,index]=scaler.inverse_transform(pred1_e2d2[:,:,index])
    pred_e2d2[:,:,index]=scaler.inverse_transform(pred_e2d2[:,:,index])
    
#     y_train[:,:,index]=scaler.inverse_transform(y_train[:,:,index])
    y_final[:,:,index]=scaler.inverse_transform(y_final_test[:,:,index])
    

In [226]:
pred_e1d1 = pred_e1d1.reshape(pred_e1d1.shape[1], pred_e1d1.shape[2])
pred_e2d2 = pred_e2d2.reshape(pred_e2d2.shape[1], pred_e2d2.shape[2])
y_final = y_final.reshape(y_final.shape[1], y_final.shape[2])

In [233]:
y_final[:,4]

array([ 80.7, 116.6, 103.3,  93.6, 100.5, 113.7, 137.7, 140.7, 169.4,
       134.9, 111.8, 166.7, 122.4, 200.5])

In [230]:
pred_e1d1[:,4]

array([105.036606, 107.506645, 108.91279 , 109.80076 , 110.3893  ,
       110.76047 , 110.95468 , 111.00233 , 110.93272 , 110.774704,
       110.55518 , 110.29776 , 110.02183 , 109.742584], dtype=float32)

In [238]:
mse = ((y_final[:,4] - pred_e2d2[:,4]) ** 2).mean()
print('RMSE: {}'.format(round(np.sqrt(mse), 2)))

RMSE: 31.33


In [ ]:
from sklearn.metrics import mean_absolute_error
for index,i in enumerate(train_df.columns):
  print(i)
  for j in range(1,6):
    print("Day ",j,":")
    print("MAE-E1D1 : ",mean_absolute_error(y_test[:,j-1,index],pred1_e1d1[:,j-1,index]),end=", ")
    print("MAE-E2D2 : ",mean_absolute_error(y_test[:,j-1,index],pred1_e2d2[:,j-1,index]))
  print()
  print()

In [2]:
df=pd.read_csv(r'household_power_consumption.txt', sep=';', header=0, low_memory=False, infer_datetime_format=True, parse_dates={'datetime':[0,1]}, index_col=['datetime'])
df.head()

,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
datetime,,,,,,,
2006-12-16 17:24:00,4.216,0.418,234.84,18.4,0.0,1.0,17.0
2006-12-16 17:25:00,5.360,0.436,233.63,23.0,0.0,1.0,16.0
2006-12-16 17:26:00,5.374,0.498,233.29,23.0,0.0,2.0,17.0
2006-12-16 17:27:00,5.388,0.502,233.74,23.0,0.0,1.0,17.0
2006-12-16 17:28:00,3.666,0.528,235.68,15.8,0.0,1.0,17.0


In [3]:
df = df.replace('?', np.nan)
df.isnull().sum()

Global_active_power      25979
Global_reactive_power    25979
Voltage                  25979
Global_intensity         25979
Sub_metering_1           25979
Sub_metering_2           25979
Sub_metering_3           25979
dtype: int64

In [14]:
def fill_missing(values):
    one_day = 60*24
    for row in range(df.shape[0]):
        for col in range(df.shape[1]):
            if np.isnan(values[row,col]):
                values = values[row-one_day,col]
df = df.astype('float32')
fill_missing(df.values)
df.isnull().sum()

IndexError: invalid index to scalar variable.

In [ ]:
daily_df = df.resample('D').sum()
daily_df.head()